In [1]:
from related_posts_generator import Post
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import umap
from pathlib import Path
from tqdm.notebook import trange, tqdm

In [7]:

def get_all_posts() -> list[Post]:
    all_posts = []
    all_posts_paths = Path.glob(Path("/home/wint3rmute/oscean/site/"), "**/*.html")
    all_posts_paths = [path for path in all_posts_paths]

    for post_path in tqdm(all_posts_paths):
        # print(post_path)
        all_posts.append(Post.from_path(post_path))

    return all_posts

all_posts = get_all_posts()

  0%|          | 0/455 [00:00<?, ?it/s]

[WARNING] Could not fetch resource ../etc/ski.html: Could not fetch ../etc/ski.html
  InvalidUrlException "../etc/ski.html" "Invalid URL"
[WARNING] Could not fetch resource ../etc/1th.c.txt: Could not fetch ../etc/1th.c.txt
  InvalidUrlException "../etc/1th.c.txt" "Invalid URL"
[WARNING] Could not fetch resource ../etc/color9.c.txt: Could not fetch ../etc/color9.c.txt
  InvalidUrlException "../etc/color9.c.txt" "Invalid URL"
[WARNING] Could not fetch resource ../etc/spacetime.s.txt: Could not fetch ../etc/spacetime.s.txt
  InvalidUrlException "../etc/spacetime.s.txt" "Invalid URL"
[WARNING] Could not fetch resource ../etc/unicycle/index.html: Could not fetch ../etc/unicycle/index.html
  InvalidUrlException "../etc/unicycle/index.html" "Invalid URL"


KeyboardInterrupt: 

In [ ]:
# all_posts[1].title
# all_posts[3].content

In [ ]:

# Vectorizer to convert a collection of raw documents to a matrix of TF-IDF features
vectorizer = TfidfVectorizer()

# Learn vocabulary and idf, return term-document matrix.
tfidf = vectorizer.fit_transform([post.content for post in all_posts])

umap_result = umap.UMAP().fit_transform(
    tfidf
)

In [ ]:
umap_result

In [ ]:

for post, post_position in zip(all_posts, umap_result):
    plt.scatter(post_position[0], post_position[1])
    # plt.annotate(post.title, post_position, post_position)


plt.show()